In [1]:
# libraries
import pyodbc # for connecting from Python to SQL
import pandas as pd # our "go-to" library for data wrangling
import numpy as np

In [6]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server={LAPTOP-8LH48O8C};'
                      'Database={AdventureWorksDW2017};'
                      'Trusted_Connection=yes;')

In [3]:
sql_for_df ="SELECT fis.[ProductKey],fis.[OrderDateKey],fis.[ProductStandardCost],fis.[UnitPrice],fis.[CustomerKey],fis.[SalesAmount],fis.[OrderQuantity],[dbo].[DimProduct].EnglishProductName,[dbo].[DimProduct].EnglishDescription,[dbo].[DimProduct].ModelName,[dbo].[DimProduct].ProductSubcategoryKey FROM [dbo].[FactInternetSales] as fis LEFT JOIN [dbo].[DimProduct]  ON [dbo].[DimProduct].[ProductKey]=fis.[ProductKey]"
rs = pd.read_sql(sql_for_df , conn)

conn.close()

<ipython-input-3-b84e42bae1d6>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rs = pd.read_sql(sql_for_df , conn)


In [7]:
sql_for_df="select * from [dbo].[ProspectiveBuyer]"
rating=pd.read_sql(sql_for_df , conn)

<ipython-input-7-1cf450cbe835>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rating=pd.read_sql(sql_for_df , conn)


In [4]:
df=pd.DataFrame(rs)

In [8]:
df1=pd.DataFrame(rating)

In [9]:
df1

,ProspectiveBuyerKey,ProspectAlternateKey,FirstName,MiddleName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,YearlyIncome,...,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,City,StateProvinceCode,PostalCode,Phone,Salutation,Unknown
0,1,21596444800,Adam,None,Alexander,1945-12-30,M,M,aalexander@lucernepublishing.com,40000.0,...,1,2,566 S. Main,None,Cedar City,UT,84720,516-555-0187,Mr.,0
1,2,3003,Adrienne,None,Alonso,1950-03-06,M,F,aalonso@alpineskihouse.com,80000.0,...,1,2,7264 St. Peter Court,None,Colma,CA,94014,607-555-0119,Ms.,4
2,3,1077,Alfredo,B,Alvarez,1964-03-30,S,M,aalvarez@fineartschool.net,130000.0,...,1,4,8850 Via Del Sol,None,Lynnwood,WA,98036,1 (11) 500 555-0143,Mr.,1
3,4,4779,Arthur,A,Arun,1985-03-22,S,M,aarun@adventure-works.com,50000.0,...,1,2,7515 Royal Oak Rd.,None,Chula Vista,CA,91910,403-555-0186,Mr.,4
4,5,38032399400,Andrea,M,Bailey,1965-04-26,M,F,abailey@lucernepublishing.com,10000.0,...,1,0,2500 North Stemmons Freeway,None,Dallas,TX,75201,1 (11) 500 555-0113,Ms.,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,2055,85344576239,Zachary,O,Brown,1966-08-02,M,M,zbrown@fineartschool.net,130000.0,...,1,3,642 Camby Rd.,None,Olympia,WA,98501,1 (11) 500 555-0140,Mr.,1
2055,2056,29688712400,Zachary,None,Gonzales,1958-10-11,M,M,zgonzales@margiestravel.com,10000.0,...,1,2,9027 Alan Dr,None,Bellingham,WA,98225,1 (11) 500 555-0133,Mr.,1
2056,2057,29740307331,Zachary,None,Li,1966-11-07,S,M,zli@adatum.com,80000.0,...,1,0,2157 Clark Creek Lane,None,Burbank,CA,91502,996-555-0189,Mr.,4
2057,2058,40882761478,Zoe,None,Stewart,1967-03-09,M,F,zstewart@consolidatedmessenger,10000.0,...,1,0,Indiana Factory Shops,None,Daleville,IN,47334,1 (11) 500 555-0188,Ms.,3


In [8]:
df1.shape

(2059, 24)

In [9]:
df["rating"]=df1["Unknown"]

In [10]:
df=df.dropna()

In [11]:
df_product=df[["rating","ProductKey","EnglishProductName","CustomerKey"]]

In [12]:
df_product

,rating,ProductKey,EnglishProductName,CustomerKey
0,0.0,310,"Road-150 Red, 62",21768
1,4.0,346,"Mountain-100 Silver, 44",28389
2,1.0,346,"Mountain-100 Silver, 44",25863
3,4.0,336,"Road-650 Black, 62",14501
4,4.0,346,"Mountain-100 Silver, 44",11003
...,...,...,...,...
2054,1.0,349,"Mountain-100 Black, 42",26112
2055,1.0,313,"Road-150 Red, 52",20626
2056,4.0,310,"Road-150 Red, 62",20812
2057,3.0,314,"Road-150 Red, 56",15669


In [13]:
combine_rating = df_product.dropna(axis=0,subset=['EnglishProductName'])
rating_count=(combine_rating.
  groupby(by=['EnglishProductName'])['rating'].
  count().
  reset_index().
  rename(columns = {'rating':'tota_rating'})
  [['EnglishProductName','tota_rating']]
             )
rating_count.head()

,EnglishProductName,tota_rating
0,"Mountain-100 Black, 38",44
1,"Mountain-100 Black, 42",42
2,"Mountain-100 Black, 44",58
3,"Mountain-100 Black, 48",52
4,"Mountain-100 Silver, 38",55


In [14]:
rating_with_totalratingcount=combine_rating.merge(rating_count,left_on='EnglishProductName',right_on='EnglishProductName',how='left')
rating_with_totalratingcount

,rating,ProductKey,EnglishProductName,CustomerKey,tota_rating
0,0.0,310,"Road-150 Red, 62",21768,312
1,4.0,346,"Mountain-100 Silver, 44",28389,47
2,1.0,346,"Mountain-100 Silver, 44",25863,47
3,4.0,336,"Road-650 Black, 62",14501,22
4,4.0,346,"Mountain-100 Silver, 44",11003,47
...,...,...,...,...,...
2054,1.0,349,"Mountain-100 Black, 42",26112,42
2055,1.0,313,"Road-150 Red, 52",20626,282
2056,4.0,310,"Road-150 Red, 62",20812,312
2057,3.0,314,"Road-150 Red, 56",15669,277


In [15]:
popularity_threshold=50
rating_popular=rating_with_totalratingcount.query('tota_rating >=@popularity_threshold')
rating_popular.head()    

,rating,ProductKey,EnglishProductName,CustomerKey,tota_rating
0,0.0,310,"Road-150 Red, 62",21768,312
5,1.0,311,"Road-150 Red, 44",27645,264
6,4.0,310,"Road-150 Red, 62",16624,312
7,4.0,351,"Mountain-100 Black, 48",11005,52
8,4.0,344,"Mountain-100 Silver, 38",11011,55


In [16]:
rating_popular.shape

(1621, 5)

In [17]:
product_feature_df=rating_popular.pivot_table(index='EnglishProductName',columns='CustomerKey',values='rating').fillna(0.0)

In [18]:
product_feature_df.head()

CustomerKey,11000,11001,11005,11009,11011,11017,11025,11030,11038,11044,...,29350,29352,29354,29385,29391,29403,29449,29466,29467,29475
EnglishProductName,,,,,,,,,,,,,,,,,,,,,
"Mountain-100 Black, 44",0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,1.0,0.0
"Mountain-100 Black, 48",0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Mountain-100 Silver, 38",1.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,4.0
"Road-150 Red, 44",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Road-150 Red, 48",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from scipy.sparse import csr_matrix

In [20]:
product_feature_df_matrix=csr_matrix(product_feature_df)

In [21]:
from sklearn.neighbors import NearestNeighbors

In [22]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(product_feature_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [23]:
query_index=np.random.choice(product_feature_df.shape[0])

In [25]:
distances,indices=model_knn.kneighbors(product_feature_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors = 6)

In [26]:
product_feature_df

CustomerKey,11000,11001,11005,11009,11011,11017,11025,11030,11038,11044,...,29350,29352,29354,29385,29391,29403,29449,29466,29467,29475
EnglishProductName,,,,,,,,,,,,,,,,,,,,,
"Mountain-100 Black, 44",0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,1.0,0.0
"Mountain-100 Black, 48",0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Mountain-100 Silver, 38",1.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,4.0
"Road-150 Red, 44",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Road-150 Red, 48",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Road-150 Red, 52",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Road-150 Red, 56",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Road-150 Red, 62",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
for i in range(0,len(distances.flatten())):
        if i == 0:
             print('reccomdations for {0}:\n'.format(product_feature_df.index[query_index]))
        else:
             print('{0}:{1},with distance of {2}:'.format(i,product_feature_df.index[indices.flatten()[i]],distances.flatten()[i]))

reccomdations for Mountain-100 Black, 48:

1:Road-150 Red, 52,with distance of 1.0:
2:Road-150 Red, 44,with distance of 1.0:
3:Road-150 Red, 56,with distance of 1.0:
4:Mountain-100 Black, 44,with distance of 1.0:
5:Mountain-100 Silver, 38,with distance of 1.0:
